In [165]:
import numpy as np
import pandas as pd
import cv2
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mostCommon import most_common
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression


In [166]:
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

In [167]:
X_train=[]
X_test=[]
y_train=[]
y_test=[]

In [168]:
df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)
X=np.asarray(images_data)
y = np.asarray(my_data['Label'])

number = preprocessing.LabelEncoder()
y=number.fit_transform(y)

In [169]:
print('Applying pca to the data')
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set
X = scaler.transform(X)
pca = PCA(n_components=270)
pca.fit(X)
X = pca.transform(X)

Applying pca to the data


KeyboardInterrupt: 

In [ ]:
l=0
ll=[0,1,2,3,4]
ll=[t+l for t in ll]
print(ll)
X_test=np.take(X,ll,axis=0)
y_test=np.take(y,ll)
X_train=np.delete(X,ll,axis=0)
y_train=np.delete(y,ll)
print(len(X_test))
print(len(y_test))
print(len(X_train))
print(len(y_train))

In [ ]:
r = []
conf_matricies = []
f1_reports = []
f1_micro = []
total_actual=[]
total_predicted=[]
l=5
ll=[-5,-4,-3,-2,-1]
for o in range(122):
    print('---------------------------- Patient ' + str(o) + ' ------------------------------ \n')
    ll=[t+l for t in ll]
    print('Indexes: '+str(ll))
    X_test=np.take(X,ll,axis=0)
    y_test=np.take(y,ll)
    X_train=np.delete(X,ll,axis=0)
    y_train=np.delete(y,ll)
    r=[]
    r.append(random.randint(0, 1000))
    print('Random seeds are: '+str(r))
    number_of_models = 11
    Xandy = []
    models = []
    # creating models
    for i in range(number_of_models):
        models.append(LogisticRegression(random_state=42).fit(X, y))

    # Creating array test splits for models
    # X_train, X_test, y_train, y_test
    for i in range(number_of_models):
        #append x train x test y train ytest to x
        x=[]
        x.append(X_train)
        x.append(X_test)
        x.append(y_train)
        x.append(y_test)
        Xandy.append(x)

    print('Equalizing Data...')
    for i in range(number_of_models):
        x = Xandy[i][0]
        y1 = Xandy[i][2]
        cov = np.count_nonzero(y1 == 0)
        pneu = np.count_nonzero(y1 == 1)
        reg = np.count_nonzero(y1 == 2)
        # deleting random pneumonia elements from each model set
        while cov != pneu:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 1:
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                pneu -= 1
        # deleting random regular elements from each model set
        while cov != reg:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 2:
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                reg -= 1
        Xandy[i][0] = x
        Xandy[i][2] = y1

    print('Training Models...')
    for i in range(number_of_models):
        models[i].fit(Xandy[i][0], Xandy[i][2])

    print('getting predictions of each model...')
    predicted = []
    for i in range(number_of_models):
        predicted.append(models[i].predict(Xandy[i][1]))

    voted_prediction = []
    for i in range(len(predicted[0])):
        voting = []
        for j in range(len(predicted)):
            voting.append(predicted[j][i])
        voted_prediction.append(most_common(voting))
   # conf_matrix = confusion_matrix(Xandy[0][3], voted_prediction)
    #print(conf_matrix)
    #conf_matricies.append(conf_matrix)

   # classification_report = metrics.classification_report(Xandy[0][3], voted_prediction, digits=3)
    #print(classification_report)
   # f1_reports.append(classification_report)
    #f1_micro.append(f1_score(Xandy[0][3], voted_prediction, average='micro'))
    total_actual.append(Xandy[0][3])
    total_predicted.append(voted_prediction)
    #print('Average f1 micro of all iterations is: ' + str(sum(f1_micro) / len(f1_micro)))
    print(voted_prediction)
    print(Xandy[0][3])
#print('Saving models....')
#np.save('models', models)
print('All Done :)')

In [ ]:
print(total_predicted)
print(total_actual)

In [ ]:
i=0
counter=0
one_prediction=[]
patient_prediction=[]
patient_test=[]
while i < len(y_test):
    one_prediction.append(voted_prediction[i])
    counter+=1
    if counter == 5:
        #append most common into final one
        patient_prediction.append(most_common(one_prediction))
        patient_test.append(Xandy[0][3][i])
        counter=0
        one_prediction=[]
    i+=1
print(patient_prediction)
print(patient_test)

In [ ]:
cm=[[0,0,0],
    [0,0,0],
    [0,0,0]]
for i in range(len(patient_prediction)):
    a=patient_prediction[i]
    b=patient_test[i]
    if a == b and a=='covid':
        cm[0][0]=cm[0][0]+1
    if a == b and a=='pneumonia':
        cm[1][1]=cm[1][1]+1
    if a == b and a=='regular':
        cm[2][2]=cm[2][2]+1
    if a=='covid' and b=='pneumonia':
        cm[0][1]=cm[0][1]+1
    if a=='covid' and b=='regular':
        cm[0][2]=cm[0][2]+1
    if a=='pneumonia' and b=='covid':
        cm[1][0]=cm[1][0]+1
    if a=='pneumonia' and b=='regular':
        cm[1][2]=cm[1][2]+1
    if a=='regular' and b=='covid':
        cm[2][0]=cm[2][0]+1
    if a=='regular' and b=='pneumonia':
        cm[2][1]=cm[2][1]+1

In [70]:
print(cm[0])
print(cm[1])
print(cm[2])

[2, 1, 1]
[0, 4, 2]
[0, 0, 12]
